In [1]:
import glob
invasive_dirs = [
    r'E:\VeligerData\invasive',
    r'E:\VeligerData\Baylor 2022-03-21_2\Veligers',
    r'E:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Zebra Ped 1 To Baylor\Preserved Zebra Ped 1 To Baylor\Sorted Images\Pedi-Zebra Veligers',
    r'E:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Zebra Ped 1a To Baylor\Preserved Zebra Ped 1a To Baylor\Sorted Images\Preserved Zebra Ped 1a',    
    r'E:\VeligerData\To Baylor 2023-01-30\To Baylor 2023-01-30\Zebra Pediveliger Image1a\Zebra Pediveligers',
  ]

# Non-Invasive category
non_invasive_dirs = [
    r'E:\VeligerData\noninvasive',
    r'E:\VeligerData\Baylor 2022-03-21_2\NonVeligers\Images_001',
    r'E:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Zebra Ped 1 To Baylor\Preserved Zebra Ped 1 To Baylor\Sorted Images\Not',
    r'E:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Zebra Ped 1a To Baylor\Preserved Zebra Ped 1a To Baylor\Sorted Images\Not',  
  ]

# Ostracod category
ostracod_dirs = [
    r'E:\VeligerData\Ostracods Day 2 Image1 To Baylor\Ostracods',
    r'E:\VeligerData\To Baylor 2023-01-30\To Baylor 2023-01-30\Ostracod Image1\Ostracods1',
]

# List to store subdirectories
invasive_subdirs = []
non_invasive_subdirs = []
ostracod_subdirs = []

# Collect subdirectories in the invasive category
for invasive_dir in invasive_dirs:
    invasive_subdirs.extend(glob.glob(invasive_dir))

# Collect subdirectories in the non-invasive category
for non_invasive_dir in non_invasive_dirs:
    non_invasive_subdirs.extend(glob.glob(non_invasive_dir))
    
for ostracod_dir in ostracod_dirs:
    ostracod_subdirs.extend(glob.glob(ostracod_dir))
    
import glob
from PIL import Image

# List of directories in x
x = non_invasive_subdirs

# List to store subdirectories
y1 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y1.extend(subdirectories)

import glob

# List of directories in x
x = invasive_subdirs

# List to store subdirectories
y2 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y2.extend(subdirectories)

    
import glob

# List of directories in x
x = ostracod_subdirs

# List to store subdirectories
y3 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y3.extend(subdirectories)

    

In [3]:

import os
import numpy as np
import tensorflow as tf
from PIL import Image
from tqdm import tqdm

def preprocess_images(y1, label_num, target_size=(40, 40)):
    # List to store image files
    image_files = []
    # List to store labels
    labels = []

    # Retrieve image files and create labels for each directory in y1
    for directory in y1:
        for root, dirs, files in os.walk(directory):
            for file in files:
                # Check if the file has an image extension
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    # Add the file path to the image_files list
                    image_files.append(os.path.join(root, file))
                    # Add the label to the labels list
                    

    # List to store preprocessed images
    images = []

    # Preprocess each image
    for file in tqdm(image_files):
        try:
            # Read the image using PIL
            image = Image.open(file)
            # Resize the image using tf.image.resize_with_crop_or_pad()
            image = tf.image.resize_with_crop_or_pad(
                tf.keras.preprocessing.image.img_to_array(image),
                target_size[0],
                target_size[1]
            )
            # Normalize the image pixels for ML training
            image = image / 255.0
            # Add the preprocessed image to the images list
            images.append(image)
            labels.append(label_num)
        except Exception as e:
            print(f"Error processing image: {file}. Skipping...")
            continue

    # Convert the images and labels lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    return images, labels


X0,Y0 = preprocess_images(y1, label_num = 0)
X1,Y1 = preprocess_images(y2, label_num = 1)
X2,Y2 = preprocess_images(y3, label_num = 2)

100%|███████████████████████████████████████████████████████████████████████| 1626/1626 [00:09<00:00, 164.85it/s]


In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Concatenate the image arrays
train_images = np.concatenate((X0, X1, X2), axis=0)

# Combine the label arrays
train_labels = np.concatenate((Y0, Y1, Y2), axis=0)

# Convert the labels to categorical
train_labels_categorical = to_categorical(train_labels)

# Set the dtype of train_images to float32
train_images = train_images.astype('float32')

# Split the data into train and test sets
from sklearn.utils import shuffle

# Shuffle the data
train_images, train_labels_categorical = shuffle(train_images, train_labels_categorical)

# Now the shuffled data is assigned to the same variable names
X_train, X_test, Y_train, Y_test = train_test_split(train_images, train_labels_categorical, test_size=0.01, random_state=42)

In [5]:
np.shape(X_train)

(139200, 40, 40, 3)

In [6]:
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), input_shape=(40, 40, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(16, (3, 3)))
model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Dense(10))
model.add(layers.Dense(3, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 38, 38, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 19, 19, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 17, 17, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 6, 16)          4624      
                                                                 
 flatten (Flatten)           (None, 576)               0

In [6]:
class_weights = np.zeros(3)

unique_labels, counts = np.unique(train_labels, return_counts=True)

total_samples = np.sum(counts)

for i, label in enumerate(unique_labels):
    class_weights[label] = total_samples / (len(unique_labels) * counts[i])

print("Class weights:", class_weights)

Class weights: [ 0.44741921  1.47039785 11.78250591]


In [7]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics='accuracy')
class_weights = [1,1,1]
model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
3480/3480 [==============================] - 58s 14ms/step - loss: 0.1625 - accuracy: 0.9402 - val_loss: 0.1449 - val_accuracy: 0.9474
Epoch 2/50
3480/3480 [==============================] - 54s 15ms/step - loss: 0.1276 - accuracy: 0.9514 - val_loss: 0.1141 - val_accuracy: 0.9568
Epoch 3/50
3480/3480 [==============================] - 36s 10ms/step - loss: 0.1169 - accuracy: 0.9552 - val_loss: 0.1100 - val_accuracy: 0.9597
Epoch 4/50
3480/3480 [==============================] - 46s 13ms/step - loss: 0.1088 - accuracy: 0.9576 - val_loss: 0.1107 - val_accuracy: 0.9577
Epoch 5/50
3480/3480 [==============================] - 51s 15ms/step - loss: 0.1041 - accuracy: 0.9592 - val_loss: 0.1016 - val_accuracy: 0.9625
Epoch 6/50
3480/3480 [==============================] - 55s 16ms/step - loss: 0.1001 - accuracy: 0.9614 - val_loss: 0.1033 - val_accuracy: 0.9615
Epoch 7/50
3480/3480 [==============================] - 28s 8ms/step - loss: 0.0966 - accuracy: 0.9626 - val_loss: 0.0966 - 

In [8]:
from tensorflow.keras.models import load_model
# Assuming you have a Keras model named "model"
model.save("full_single_image.h5")

In [28]:
from tensorflow.keras.models import load_model

# Define the path to the model file
model_path = r'E:\ML_notebooks\premodel.h5'

# Load the model with custom_objects argument
#model = load_model(model_path)

In [29]:
invasive_dirs = [
    r'D:\VeligerData\To Baylor 2023-01-30\To Baylor 2023-01-30\Zebra Pediveliger Image1a\Zebra Pediveligers',
    r'E:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1 Image1\Baylor Preserved Zebra Umbo 1 Image1\Sorted Images\Umbonal',
    r'E:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1a Image1\Baylor Preserved Zebra Umbo 1a Image1\Sorted Images\Umbonal',
    r'E:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Preserved Zebra D-Hinge 1 Baylor\Preserved Zebra D-Hinge 1 Baylor\Sorted Images\Zebra D-Hinge',
    r'C:\Users\shaif\Downloads\Compressed\Labeled Baylor 8-30-23\Labeled Baylor 8-30-23\Zebra Ped 1a Image1a\Pedi',
    r'C:\Users\shaif\Downloads\Compressed\Labeled Baylor 8-30-23\Labeled Baylor 8-30-23\Zebra Ped 1a Image1a\D-Hinge'
]

# Non-Invasive category
non_invasive_dirs = [
    r'D:\VeligerData\To Baylor 2023-01-30\To Baylor 2023-01-30\Not Veligers\Z-P',
    r'E:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1 Image1\Baylor Preserved Zebra Umbo 1 Image1\Sorted Images\Not',
    r'E:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1a Image1\Baylor Preserved Zebra Umbo 1a Image1\Sorted Images\Not',
    r'E:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Preserved Zebra D-Hinge 1 Baylor\Preserved Zebra D-Hinge 1 Baylor\Sorted Images\Not',
    r'D:\2.Veliger_Data\1.Samples 1 - 3\Samples 1 - 3\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image2a\Images_001'
]

# Ostracod category
ostracod_dirs = [
    r'D:\VeligerData\Ostracod Day 2 Image12 Short To Baylor\Ostracod Day 2 Image12 To Baylor\Sorted Images\Ostracods',
    r'E:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Preserved Zebra D-Hinge 1 Baylor\Preserved Zebra D-Hinge 1 Baylor\Sorted Images\Ostracod',
    r'E:\VeligerData\Ostracod Day 2 Image12 Short To Baylor\Ostracod Day 2 Image12 To Baylor\Sorted Images\Ostracods',
    r'E:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Ostracods 1 To Baylor\Preserved Ostracods 1 To Baylor\Sorted Images\Preserve Ostracods',
    r'E:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Ostracods 1a To Baylor\Preserved Ostracods 1a To Baylor\Sorted Images\Preserved Ostracods 1a',
    r'E:\VeligerData\Ostracods Day 2 Image3 To Baylor_2\Ostracods Day 2 Image3 To Baylor\Sorted Images\Ostracods',
    r'E:\VeligerData\Ostracods Day 2 Image12 To Baylor_3\Ostracods Day 2 Image12 To Baylor\Sorted Images\Ostracods',  

  ]

# List to store subdirectories
invasive_subdirs = []
non_invasive_subdirs = []
ostracod_subdirs = []

# Collect subdirectories in the invasive category
for invasive_dir in invasive_dirs:
    invasive_subdirs.extend(glob.glob(invasive_dir))

# Collect subdirectories in the non-invasive category
for non_invasive_dir in non_invasive_dirs:
    non_invasive_subdirs.extend(glob.glob(non_invasive_dir))
    
for ostracod_dir in ostracod_dirs:
    ostracod_subdirs.extend(glob.glob(ostracod_dir))

In [30]:
import glob
from PIL import Image

# List of directories in x
x = non_invasive_subdirs

# List to store subdirectories
y1 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y1.extend(subdirectories)

import glob

# List of directories in x
x = invasive_subdirs

# List to store subdirectories
y2 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y2.extend(subdirectories)

    
import glob

# List of directories in x
x = ostracod_subdirs

# List to store subdirectories
y3 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y3.extend(subdirectories)

    

    
import os
import numpy as np
import tensorflow as tf
from PIL import Image

def preprocess_images(y1, label_num, target_size=(40, 40)):
    # List to store image files
    image_files = []
    # List to store labels
    labels = []

    # Retrieve image files and create labels for each directory in y1
    for directory in y1:
        for root, dirs, files in os.walk(directory):
            for file in files:
                # Check if the file has an image extension
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    # Add the file path to the image_files list
                    image_files.append(os.path.join(root, file))
                    # Add the label to the labels list
                    

    # List to store preprocessed images
    images = []

    # Preprocess each image
    for file in tqdm(image_files):
        try:
            # Read the image using PIL
            image = Image.open(file)
            # Resize the image using tf.image.resize_with_crop_or_pad()
            image = tf.image.resize_with_crop_or_pad(
                tf.keras.preprocessing.image.img_to_array(image),
                target_size[0],
                target_size[1]
            )
            # Normalize the image pixels for ML training
            image = image / 255.0
            # Add the preprocessed image to the images list
            images.append(image)
            labels.append(label_num)
        except Exception as e:
            print(f"Error processing image: {file}. Skipping...")
            continue

    # Convert the images and labels lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    return images, labels


X0,Y0 = preprocess_images(y1, label_num = 0)
X1,Y1 = preprocess_images(y2, label_num = 1)
X2,Y2 = preprocess_images(y3, label_num = 2)

Error processing image: D:\VeligerData\Ostracod Day 2 Image12 Short To Baylor\Ostracod Day 2 Image12 To Baylor\Sorted Images\Ostracods\Object_002\._Image_032.png. Skipping...
Error processing image: D:\VeligerData\Ostracod Day 2 Image12 Short To Baylor\Ostracod Day 2 Image12 To Baylor\Sorted Images\Ostracods\Object_002\._Image_036.png. Skipping...


In [31]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Concatenate the image arrays
train_images = np.concatenate((X0, X1, X2), axis=0)

# Combine the label arrays
train_labels = np.concatenate((Y0, Y1, Y2), axis=0)

# Convert the labels to categorical
train_labels_categorical = to_categorical(train_labels)

# Set the dtype of train_images to float32
train_images = train_images.astype('float32')


# Split the data into train and test sets
from sklearn.utils import shuffle

# Assuming you have the original train_images and train_labels_categorical

# Shuffle the data
train_images, train_labels_categorical = shuffle(train_images, train_labels_categorical)

# Now the shuffled data is assigned to the same variable names

X_train, X_test1, Y_train, Y_test1 = train_test_split(train_images, train_labels_categorical, test_size=0.01, random_state=42)


In [12]:
from tensorflow.keras.models import load_model
# Assuming you have a Keras model named "model"
model.save("premodel.h5")

In [32]:
np.shape(X_train)

(62657, 40, 40, 3)

In [33]:
class_weights = np.zeros(3)

unique_labels, counts = np.unique(train_labels, return_counts=True)

total_samples = np.sum(counts)

for i, label in enumerate(unique_labels):
    class_weights[label] = total_samples / (len(unique_labels) * counts[i])

print("Class weights:", class_weights)

Class weights: [0.51307619 1.11912719 6.35250427]


In [34]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics='accuracy')
class_weights = [1,1,1]
model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

Epoch 1/50
1763/1763 [==============================] - 19s 11ms/step - loss: 0.3414 - accuracy: 0.8741 - val_loss: 0.3145 - val_accuracy: 0.8726
Epoch 2/50
1763/1763 [==============================] - 18s 10ms/step - loss: 0.2565 - accuracy: 0.8975 - val_loss: 0.2602 - val_accuracy: 0.8974
Epoch 3/50
1763/1763 [==============================] - 19s 11ms/step - loss: 0.2364 - accuracy: 0.9075 - val_loss: 0.2490 - val_accuracy: 0.8995
Epoch 4/50
1763/1763 [==============================] - 15s 9ms/step - loss: 0.2257 - accuracy: 0.9107 - val_loss: 0.2619 - val_accuracy: 0.8974
Epoch 5/50
1763/1763 [==============================] - 17s 10ms/step - loss: 0.2126 - accuracy: 0.9161 - val_loss: 0.2389 - val_accuracy: 0.9055
Epoch 6/50
1763/1763 [==============================] - 17s 9ms/step - loss: 0.2043 - accuracy: 0.9189 - val_loss: 0.2602 - val_accuracy: 0.9015
Epoch 7/50
1763/1763 [==============================] - 18s 10ms/step - loss: 0.1962 - accuracy: 0.9217 - val_loss: 0.3115 - v

Accuracy: 1.0


In [36]:
from tensorflow.keras.models import load_model
# Assuming you have a Keras model named "model"
model.save("single_image_model.h5")